<a href="https://colab.research.google.com/github/KUNALOP16857/visual-product-measurement-system/blob/main/visual_product_measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch torchvision transformers pandas openpyxl pillow reportlab requests



In [18]:
!pip install opencv-python


In [43]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_p3jabpp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_p3jabpp
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=fa0f98b40dd8314368d2ffe912393465a8a5852f415d55df1817a7237c8f106d
  Stored in directory: /tmp/pip-ephem-wheel-cache-vbt4uqba/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [44]:
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()


100%|███████████████████████████████████████| 338M/338M [00:12<00:00, 28.0MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [2]:
import pandas as pd
import torch
import requests
import json
from PIL import Image
from io import BytesIO
from typing import List
from pydantic import BaseModel
from transformers import BlipProcessor, BlipForConditionalGeneration



In [4]:
from google.colab import files
files.upload()  # Upload A1.0_data_product_images.xlsx


Saving A1.0_data_product_images.xlsx to A1.0_data_product_images (1).xlsx


{'A1.0_data_product_images (1).xlsx': b'PK\x03\x04\x14\x00\x08\x08\x08\x00\x1c\x1e\x9a[\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\x00\x00\x00xl/drawings/drawing1.xml\x9d\xd0]n\xc20\x0c\x07\xf0\x13\xec\x0eU\xdeiZ\x18\x13C\x14^\xd0N0\x0e\xe0%n\x1b\x91\x8f\xca\x0e\xa3\xdc~\xd1J6i{\x01\x1em\xcb?\xf9\xef\xcdnt\xb6\xf8Db\x13|#\xea\xb2\x12\x05z\x15\xb4\xf1]#\x0e\xefo\xb3\x95(8\x82\xd7`\x83\xc7F\\\x90\xc5n\xfb\xb4\x195\xad\xcf\xbc\xa7"\xed{^\xa7\xb2\x11}\x8c\xc3ZJV=:\xe02\x0c\xe8\xd3\xb4\r\xe4 \xa6\x92:\xa9\t\xceIvV\xce\xab\xeaE\xf2@\x08\x9a{\xc4\xb8\x9f&\xe2\xea\xc1\x03\x9a\x03\xe3\xf3\xfeM\xd7\x84\xb65\n\xf7A\x9d\x1c\xfa8!\x84\x16b\xfa\x05\xf7f\xe0\xac\xa9\x07\xaeQ=P\xfc\x01\xc6\x7f\x823\x8a\x02\x876\x96*\xb8\xeb)\xd9HB\xfd<\t8\xfe\x1a\xf5\xdd\xc8R\xbe\xca\xd5_\xc8\xdd\x14\xc7\x01\x1dO\xc3,\xb9Cz\xc8\x87\xb1&^\xbe\x93eFw\xee\x81\xb7h\x03\x1d\x81\xcb\xc8\xb88\xf8\xe3\xdd\xb1*\xc96\xb5(+l\xb1^\xde\xad\xcc\xb3"\xb7_PK\x07\x08\x07bi\x83\x05\x01\x00\x00\x07\x03\x00\x00PK\x03\x04\x14\x00

In [5]:
df = pd.read_excel("A1.0_data_product_images.xlsx")
df.columns = df.columns.str.strip()
df.head()


,Product Id,Category,Image Count,Image1,Image2,Image3,Image4,Image5,Image6,Image7,Image8,Image9,Image10,Image11,Image12,Image13,Image14
0,231031,Eyeglasses,5,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,237793,Eyeglasses,6,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,231027,Eyeglasses,5,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,237372,Eyeglasses,7,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,231002,Eyeglasses,5,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,https://static5.lenskart.com/media/catalog/pro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.columns


Index(['Product Id', 'Category', 'Image Count', 'Image1', 'Image2', 'Image3',
       'Image4', 'Image5', 'Image6', 'Image7', 'Image8', 'Image9', 'Image10',
       'Image11', 'Image12', 'Image13', 'Image14'],
      dtype='object')

In [6]:
image_columns = [
    c for c in df.columns
    if c.lower().startswith("image") and c.lower() != "image count"
]
image_columns


['Image1',
 'Image2',
 'Image3',
 'Image4',
 'Image5',
 'Image6',
 'Image7',
 'Image8',
 'Image9',
 'Image10',
 'Image11',
 'Image12',
 'Image13',
 'Image14']

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained(
    "Salesforce/blip-image-captioning-base"
)

model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(device)

model.eval()



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [9]:
def load_image_from_url(url):
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        return Image.open(BytesIO(response.content)).convert("RGB")
    except:
        return None


In [10]:
def analyze_image_with_blip(image: Image.Image) -> str:
    inputs = processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=30)
    return processor.decode(out[0], skip_special_tokens=True)



In [11]:
class VisualScores(BaseModel):
    gender_expression: float
    visual_weight: float
    embellishment: float
    unconventionality: float
    formality: float

class VisualAttributes(BaseModel):
    wirecore: bool
    frame_geometry: str
    transparency: str
    dominant_colors: List[str]
    textures: List[str]
    suitable_for_kids: bool

class ProductAnalysis(BaseModel):
    scores: VisualScores
    attributes: VisualAttributes
    confidence: float
    notes: str


In [27]:
import cv2
import numpy as np

def extract_basic_visual_features(image: Image.Image):
    img = np.array(image)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Threshold to isolate frame structure
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    if not contours:
        return {
            "aspect_ratio": None,
            "edge_density": 0,
            "brightness": gray.mean()
        }

    # Largest contour assumed to be frame
    largest = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest)

    aspect_ratio = w / h if h > 0 else 1.0

    edges = cv2.Canny(gray, 50, 150)
    edge_density = edges.mean()

    return {
        "aspect_ratio": aspect_ratio,
        "edge_density": edge_density,
        "brightness": gray.mean()
    }


In [45]:
def clip_similarity(image: Image.Image, text_prompts: list) -> dict:
    image_input = clip_preprocess(image).unsqueeze(0).to(device)
    text_inputs = clip.tokenize(text_prompts).to(device)

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features = clip_model.encode_text(text_inputs)

        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        similarities = (image_features @ text_features.T).squeeze(0)

    return {
        text_prompts[i]: similarities[i].item()
        for i in range(len(text_prompts))
    }


In [46]:
def infer_dominant_color_with_clip(image):
    prompts = [
        "black eyeglasses",
        "brown eyeglasses",
        "clear eyeglasses",
        "colored eyeglasses"
    ]

    scores = clip_similarity(image, prompts)
    return max(scores, key=scores.get).split()[0]


In [47]:
def infer_gender_expression_with_clip(image):
    prompts = [
        "masculine eyeglasses",
        "unisex eyeglasses",
        "feminine eyeglasses"
    ]

    scores = clip_similarity(image, prompts)

    return round(
        (-1.5 * scores["masculine eyeglasses"]
         + 0.0 * scores["unisex eyeglasses"]
         + 1.5 * scores["feminine eyeglasses"]),
        2
    )


In [52]:
def infer_frame_geometry_with_clip(image, aspect_ratio):
    prompts = [
        "round eyeglasses",
        "rectangular eyeglasses",
        "square eyeglasses",
        "cat-eye eyeglasses"
    ]

    scores = clip_similarity(image, prompts)

    # Use CLIP primarily, OpenCV as fallback
    geometry = max(scores, key=scores.get).split()[0]

    if geometry == "rectangular" and aspect_ratio < 1.1:
        return "round"

    return geometry


In [62]:
def caption_to_measurements(
    caption: str,
    product_id,
    frame_geometry: str,
    dominant_color: str,
    texture: str,
    brightness: float,
    edge_density: float,
    gender_expression_override: float,
    transparency_override: str
) -> ProductAnalysis:
    """
    Converts multi-source vision signals into structured visual measurements.
    Reasoning is based exclusively on observable appearance.
    """

    caption_lower = caption.lower()

    # -----------------------------
    # 1. VISUAL MEASUREMENT SCORES
    # -----------------------------

    # --- Gender Expression (from CLIP override) ---
    gender_expression = scale_to_range(gender_expression_override)


    # --- Visual Weight: sleek/light (-) → bold/heavy (+) ---
    visual_weight = 0.0
    if texture == "textured":
        visual_weight += 1.2
    if "thick" in caption_lower:
        visual_weight += 1.0
    if "thin" in caption_lower:
        visual_weight -= 1.0
    if frame_geometry in ["square"]:
        visual_weight += 0.5
    if texture == "smooth":
        visual_weight -= 0.5

    # --- Embellishment: simple (-) → ornate (+) ---
    embellishment = -1.0
    if texture == "textured":
        embellishment += 1.5
    if "decorative" in caption_lower or "ornate" in caption_lower:
        embellishment += 1.0

    # --- Unconventionality: classic (-) → avant-garde (+) ---
    unconventionality = 0.0
    if frame_geometry == "cat-eye":
        unconventionality += 2.0
    if dominant_color not in ["black", "clear"]:
        unconventionality += 1.0
    if frame_geometry == "square":
        unconventionality += 0.5

    # --- Formality: casual (-) → formal (+) ---
    formality = 0.0
    if "metal" in caption_lower:
        formality += 1.5
    if frame_geometry in ["rectangular", "square"]:
        formality += 0.5
    if dominant_color == "clear":
        formality -= 0.5

    # Clamp helper
    def clamp(x):
        return max(-5.0, min(5.0, round(x, 2)))

    scores = VisualScores(
        gender_expression=clamp(gender_expression),
        visual_weight=clamp(visual_weight),
        embellishment=clamp(embellishment),
        unconventionality=clamp(unconventionality),
        formality=clamp(formality)
    )

    # -----------------------------
    # 2. OBSERVABLE ATTRIBUTES
    # -----------------------------

    attributes = VisualAttributes(
        wirecore="metal" in caption_lower,
        frame_geometry=frame_geometry,
        transparency=transparency_override,
        dominant_colors=[dominant_color],
        textures=[texture],
        suitable_for_kids=(
            scores.visual_weight <= 0.0 and
            scores.embellishment < 0 and
            frame_geometry in ["round", "rectangular"]
        )
    )

    # -----------------------------
    # 3. CONFIDENCE ESTIMATION
    # -----------------------------

    confidence = 0.85

    # Reduce confidence if signals are weak or ambiguous
    if frame_geometry == "unknown":
        confidence -= 0.10
    if abs(scores.gender_expression) < 0.2:
        confidence -= 0.05
    if dominant_color == "unknown":
        confidence -= 0.05

    confidence = round(max(0.5, min(0.95, confidence)), 2)

    # -----------------------------
    # 4. FINAL OUTPUT
    # -----------------------------

    return ProductAnalysis(
        scores=scores,
        attributes=attributes,
        confidence=confidence,
        notes="Measurements derived exclusively from observable visual characteristics using vision-based reasoning."
    )


In [56]:
def infer_transparency(brightness: float) -> str:
    """
    Infers lens transparency using average image brightness.
    Purely vision-based (pixel-level).
    """
    if brightness > 180:
        return "transparent"
    elif brightness > 130:
        return "semi-transparent"
    else:
        return "opaque"


In [57]:
print(infer_transparency(200))
print(infer_transparency(150))
print(infer_transparency(80))


transparent
semi-transparent
opaque


In [63]:
results = []
MAX_PRODUCTS = 10   # keep small for stability

for _, row in df.head(MAX_PRODUCTS).iterrows():

    product_id = row["Product Id"]
    print(f"\nStarting product {product_id}")

    # ---- Load images safely ----
    urls = [row[c] for c in image_columns if isinstance(row[c], str)]
    images = [load_image_from_url(u) for u in urls]
    images = [img for img in images if img is not None]

    if not images:
        print("No valid images, skipping")
        continue

    image = images[0]

    # ---- BLIP ----
    caption = analyze_image_with_blip(image)
    print("Caption:", caption)

    # ---- Pixel features ----
    features = extract_basic_visual_features(image)

    # ---- CLIP reasoning ----
    frame_geometry = infer_frame_geometry_with_clip(
        image=image,
        aspect_ratio=features.get("aspect_ratio")
    )

    dominant_color = infer_dominant_color_with_clip(image)
    gender_expression = infer_gender_expression_with_clip(image)

    transparency = infer_transparency(features["brightness"])
    texture = "smooth" if features["edge_density"] < 30 else "textured"

    # ---- Final reasoning ----
    analysis = caption_to_measurements(
        caption=caption,
        product_id=product_id,
        frame_geometry=frame_geometry,
        dominant_color=dominant_color,
        texture=texture,
        brightness=features["brightness"],
        edge_density=features["edge_density"],
        gender_expression_override=gender_expression,
        transparency_override=transparency
    )

    results.append({
        "product_id": product_id,
        "image_count": 1,
        "analysis": analysis.model_dump()
    })

    print(
    f"Processed {product_id} | "
    f"Geometry={frame_geometry}, "
    f"Color={dominant_color}, "
    f"Gender(raw)={gender_expression:.3f}, "
    f"Gender(scaled)={analysis.scores.gender_expression}"
)




Starting product 231031
Caption: the glasses are made from wood and metal
Processed 231031 | Geometry=rectangular, Color=brown, Gender(raw)=-0.030, Gender(scaled)=-3.0

Starting product 237793
Caption: a pair of glasses with black frames
Processed 237793 | Geometry=rectangular, Color=black, Gender(raw)=-0.030, Gender(scaled)=-3.0

Starting product 231027
Caption: a pair of glasses with a brown and black frame
Processed 231027 | Geometry=rectangular, Color=brown, Gender(raw)=-0.040, Gender(scaled)=-4.0

Starting product 237372
Caption: the glasses are clear and clear
Processed 237372 | Geometry=rectangular, Color=clear, Gender(raw)=-0.030, Gender(scaled)=-3.0

Starting product 231002
Caption: a pair of glasses on a white background
Processed 231002 | Geometry=rectangular, Color=black, Gender(raw)=-0.040, Gender(scaled)=-4.0

Starting product 231006
Caption: a pair of glasses on a white background
Processed 231006 | Geometry=rectangular, Color=black, Gender(raw)=-0.030, Gender(scaled)=-

In [61]:
def scale_to_range(
    value: float,
    input_min: float = -0.05,
    input_max: float = 0.05,
    output_min: float = -5.0,
    output_max: float = 5.0
) -> float:
    """
    Scales small similarity scores into a human-interpretable range.
    """

    # Clamp input
    value = max(input_min, min(input_max, value))

    # Normalize to 0–1
    normalized = (value - input_min) / (input_max - input_min)

    # Scale to output
    scaled = output_min + normalized * (output_max - output_min)

    return round(scaled, 2)


In [60]:
with open("results.json", "w") as f:
    json.dump(results, f, indent=2)

from google.colab import files
files.download("results.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>